Demonstrate making and applying calibration of phase and amplitude for a scan.

In general, the "calbration" data are calculated with a direct loop-back and then are applied to data collected while connected to actual resonators.  In these examples, we retain the loop back connection for both the calibration and the data.  This should result in "flat" scans in amplitude and phase.

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
import pickle
soc = TopSoc()

In [ ]:
fNyIn = soc.fsIn/2
print("fNyIn =",fNyIn)

In [ ]:
# Define the area to calibrate, based on an (arbitrary) mixer setting
fMixer = 0.75*soc.fsIn
print("fMixer =",fMixer)
soc.set_mixer(fMixer)
f0 = fMixer + 85.4
f1 = fMixer + 95.6
fTone = 0.5*(f0+f1)

In [ ]:
# Measure the nominal delay near fTone
scan = Scan.Scan(soc)
outCh = scan.soc.outFreq2ch(fTone)
print("measure nominal dealy for fTone=%f  outCh=%d"%(fTone, outCh))
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, doProgress=True, doPlot=True)

In [ ]:
# Make a calibration with nt transfers at nf frequencies, with the nominalDelay correction applied
nt,nf = 1,201
calibration = scan.makeCalibration(fMixer, f0, f1, nf=nf, nt=nt, doProgress=True,
                            nominalDelay=nominalDelay)
fscan = calibration['fscan']
print("delayApplied =",fscan['delayApplied'])


In [ ]:
# Save this calibration for use in the next tutorial
with open("calibration-demo.pkl", 'wb') as file:
    pickle.dump(calibration, file)

In [ ]:
# Take a look at part of the calibration data, for one of the PFB bins
iTone = 2
Scan.fscanPlot(fscan, iTone)


In [ ]:
# For convenience, arrange these points as a single spectrum
spectrum = Scan.fscanToSpectrum(fscan)

In [ ]:
# These frequencies define where there will be discontinuities in I,Q values
importlib.reload(Scan)
scan = Scan.Scan(soc)
fList = scan.makeFList(fMixer, f0, f1,verbose=True)
print(fList)

In [ ]:
# Plot the amplitudes, along with vertical lines showing where discontinuities are.  The horizontal green line shows the frequency range requested
plt.plot(spectrum[0],spectrum[1])
for f in fList:
    plt.axvline(f,color='r', alpha=0.4)
aMean = spectrum[1].mean()
plt.plot([f0,f1],[aMean,aMean], 'g')
plt.xlabel("frequency (MHz)")

In [ ]:
# Plot the phases, along with vertical lines showing where discontinuities are
plt.plot(spectrum[0],spectrum[2])
print("f0 =",f0)
print("f1 =",f1)
for f in fList:
    plt.axvline(f,color='r', alpha=0.5)
pMean = spectrum[2].mean()
plt.plot([f0,f1],[pMean, pMean], 'g')
plt.xlabel("frequency (MHz)")

In [ ]:
# zoom in a bit and show I, Q values and discontinuities
fMiddle = 0.5*(f0+f1)
fStart = fMiddle-2.3
fEnd = fMiddle+1.4
def iqPlot(fStart, fEnd):
    inds = (spectrum[0] > fStart ) & (spectrum[0] < fEnd)
    sx = spectrum[1]*np.exp(1j*spectrum[2])
    si = np.real(sx)
    sq = np.imag(sx)
    plt.plot(spectrum[0][inds],si[inds], ',', label="I")
    plt.plot(spectrum[0][inds],sq[inds], ',', label="Q")
    for f in fList:
        plt.axvline(f,color='r', alpha=0.2)
    plt.xlim((fStart,fEnd))
    plt.legend()
    plt.xlabel("frequency (MHz)")
    plt.ylabel("values [ADUs]")
iqPlot(fStart, fEnd)

In [ ]:
# Pretend we have a few frequencies we want to study.  Define tones of equal amplitude and random phases and scan
nTones = 4
testFreqs = fStart + 2.345*np.arange(nTones)

testAmps = 0.9*np.ones(len(testFreqs))/len(testFreqs)
np.random.seed(1234991)
testFis = np.random.uniform(0, 2*np.pi, size=len(testFreqs))
bandwidth = 0.4 # MHz
nf = 50
decimation = 2
nt = 3
doProgress = True

testFScan = scan.fscan(testFreqs, testAmps, testFis, bandwidth, nf, decimation,nt, doProgress=doProgress)


In [ ]:
# Plot the measured amplitude,phase of one of the tones
Scan.fscanPlot(testFScan, 0)

In [ ]:
# Apply the calibration
tfsCalib = scan.applyCalibration(testFScan, calibration, amplitudeMax=5000)


In [ ]:
# Plot the calibrated amplitude,phase of one of the tones
Scan.fscanPlot(tfsCalib, 0)

In [ ]:
Scan.fscanPlot(tfsCalib, 1)

In [ ]:
Scan.fscanPlot(tfsCalib, 2)

In [ ]:
Scan.fscanPlot(tfsCalib, 3)

In [ ]:
iqPlot(3164, 3166)

In [ ]:
def apPlot(fStart, fEnd):
    inds = (spectrum[0] > fStart ) & (spectrum[0] < fEnd)
    amp = spectrum[1]
    pha = spectrum[2]
    fig,ax = plt.subplots(2,1,sharex=True)
    
    ax[0].plot(spectrum[0][inds],amp[inds], ',')
    plt.plot(spectrum[0][inds],pha[inds], ',')
    for f in fList:
        ax[0].axvline(f,color='r', alpha=0.2)
        ax[1].axvline(f,color='r', alpha=0.2)
    ax[1].set_xlabel("frequency (MHz)")
    ax[0].set_ylabel("amplitude [ADUs]")
    ax[1].set_ylabel("phase [rad]")
    plt.xlim((fStart,fEnd))


In [ ]:
apPlot(3164, 3166)

In [ ]:
iToneTest = 2
import copy
def applyCalibration2(self, fscan, calibration, amplitudeMax=30000):
    """
    Apply the calibration to the frequency scan

    Parameters:
    -----------
        fscan : object
            returned from the function fscan.
        calibration : object
            returned from the function makeCalibration
        amplitudeMax : float
            amplitude of measurement when amplitudes of fscan and calibration are equal

    Returns:
    --------
        a deep copy of fscan with the calibration applied.
    """
    fscanCalib = copy.deepcopy(fscan)
    nominalDelay = calibration['nominalDelay']
    self.applyDelay(fscanCalib, nominalDelay)
    if nominalDelay != fscanCalib['delayApplied']:
        raise ValueError("fscan already had a delay applied", nominalDelay, fscanCalib['delayApplied'])
    dfs = fscanCalib['dfs']
    for iTone,(freq,xs) in enumerate(zip(fscanCalib['freqs'],fscanCalib['xs'])):
        print("iTone, freq",iTone,freq)
        freqs = freq+dfs
        xCalib = np.zeros(len(freqs), dtype=complex)
        for i, freq in enumerate(freqs):
            iCalib = np.searchsorted(calibration['fList'], freq)-1 
            xCalib[i] = calibration['cInterps'][iCalib](freq)
            if iTone==2:
                xi = np.real(xCalib[i])
                xq = np.imag(xCalib[i])
                #print("   i=%2d   freq=%.2f   iCalib=%d  xi=%f xq=%f"%(i,freq,iCalib,xi,xq))
        gain = amplitudeMax/np.abs(xCalib)
        fscanCalib['xs'][:,iTone] *= gain
        dfi = np.angle(xCalib)
        xs = fscanCalib['xs'][:,iTone]
        if iTone == iToneTest:
            plt.plot(freq+dfs, dfi, '.', label='dfi')
            plt.plot(freq+dfs, np.angle(xs), '.', label='xs')
            plt.legend()
        fscanCalib['xs'][:,iTone] = np.abs(xs)*np.exp(1j*(np.angle(xs)-dfi))
    return fscanCalib                                                     

# Apply the calibration
print("iToneTest =",iToneTest)
calibrated = applyCalibration2(scan, testFScan, calibration, amplitudeMax=5000)
Scan.fscanPlot(calibrated, iToneTest)

In [ ]:
phis = np.angle(calibrated['xs'][:,iToneTest])
plt.plot(np.diff(phis))
print(np.diff(phis).min())
print(np.diff(phis).min()/np.pi)


In [ ]:
import copy
def applyCalibration3(self, fscan, calibration, amplitudeMax=30000):
    """
    Apply the calibration to the frequency scan

    Parameters:
    -----------
        fscan : object
            returned from the function fscan.
        calibration : object
            returned from the function makeCalibration
        amplitudeMax : float
            amplitude of measurement when amplitudes of fscan and calibration are equal

    Returns:
    --------
        a deep copy of fscan with the calibration applied.
    """
    fscanCalib = copy.deepcopy(fscan)
    nominalDelay = calibration['nominalDelay']
    self.applyDelay(fscanCalib, nominalDelay)
    if nominalDelay != fscanCalib['delayApplied']:
        raise ValueError("fscan already had a delay applied", nominalDelay, fscanCalib['delayApplied'])
    dfs = fscanCalib['dfs']
    for iTone,(freq,xs) in enumerate(zip(fscanCalib['freqs'],fscanCalib['xs'])):
        print("iTone, freq",iTone,freq)
        freqs = freq+dfs
        xCalib = np.zeros(len(freqs), dtype=complex)
        for i, freq in enumerate(freqs):
            iCalib = np.searchsorted(calibration['fList'], freq)-1 
            xCalib[i] = calibration['cInterps'][iCalib](freq)
        
        gain = amplitudeMax/np.abs(xCalib)
        fscanCalib['xs'][:,iTone] *= gain
        xs = fscanCalib['xs'][:,iTone]
        xsReal = np.real(xs)/np.abs(xs)
        xsImag = np.imag(xs)/np.abs(xs)
        
        xcalibReal = np.real(xCalib)/np.abs(xCalib)
        xcalibImag = np.imag(xCalib)/np.abs(xCalib)
        
        xcReal = xsReal - xcalibReal
        xcImag = xsImag - xcalibImag
        fscanCalib['xs'][:,iTone] = fscanCalib['xs'][:,iTone] / gain * (xcReal + 1j*xcImag)
        if iTone == 2:
            fscan = fscanCalib['xs'][:,iTone]
            plt.plot(freq+dfs, xsReal, '.', label='xsReal')
            plt.plot(freq+dfs, xcalibReal, '.', label='xcalibReal')
            plt.legend()
    return fscanCalib                                                     

# Apply the calibration
calibrated = applyCalibration3(scan, testFScan, calibration, amplitudeMax=5000)


In [ ]:
Scan.fscanPlot(calibrated, 2)